Experiment: Anwendung von Azure-Open AI in Verbindung mit Langchain.

Bei der Nutzung von Azure-Open AI ist es erforderlich, sicherheitskritische Informationen wie API-Token zu verwenden. Um deren Sicherheit zu gewährleisten und eine Speicherung in Dateien zu vermeiden, empfiehlt sich die Verwendung von Umgebungsvariablen. Open AI stellt diese bereits vordefiniert bereit.

``` bash
# The API version you want to use: set this to `2023-12-01-preview` for the released version.
export OPENAI_API_VERSION=2024-02-15-preview
# The base URL for your Azure OpenAI resource.  You can find this in the Azure portal under your Azure OpenAI resource.
export AZURE_OPENAI_ENDPOINT=https://your-resource-name.openai.azure.com
# The API key for your Azure OpenAI resource.  You can find this in the Azure portal under your Azure OpenAI resource.
export AZURE_OPENAI_API_KEY=<your Azure OpenAI API key>
``` 

Um die ständige Neueingabe dieser Informationen zu vermeiden, können diese Befehle in die Datei ~/.bashrc eingefügt werden.

Quellen:
- https://python.langchain.com/docs/integrations/llms/azure_openai


In [54]:
import os

from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_deployment = "gpt-4"
)

message = HumanMessage(
    content="Verfasse eine 50 Wörter umfassende Vorlesegeschichte für Kinder, in der es um eine kleine Kartoffel namens Pellemann geht, die einen Hügel hinunterrollt, um ihre Freunde zu suchen."
)
model.invoke([message])

# Experiment: AzureOpenAI mit Streaming-Function
Manchmal kann die Datenmenge so groß sein, dass die Verarbeitung und Ausgabe der Ergebnisse erhebliche Zeit in Anspruch nimmt. 
Für solche Situationen ist es vorteilhaft, bereits empfangene und verarbeitete Daten fortlaufend auszugeben.

In [ ]:
from langchain.callbacks import get_openai_callback
from langchain.callbacks.base import BaseCallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


model = AzureChatOpenAI(
    azure_deployment = "gpt-4",
    streaming=True,
    callback_manager=BaseCallbackManager([
        StreamingStdOutCallbackHandler()]),
)

with get_openai_callback() as cb:
    message = HumanMessage(
        content="Verfasse eine 50 Wörter umfassende Vorlesegeschichte für Kinder, in der es um eine kleine Kartoffel namens Pellemann geht, die einen Hügel hinunterrollt, um ihre Freunde zu suchen."
    )
    model.invoke([message])    

Einmal da war eine kleine Kartoffel namens Pellemann. Eines Tages kullerte er den Hügel runter, hinaus ins weite Feld. Er hüpfte vorbei an bunten Blumen und summenden Bienen. Am Fuße des Hügels fand er endlich seine Freunde. "Hallo, Pellemann!" riefen sie und spielten den ganzen Tag zusammen.

# Expereriment AzureOpenAI mit Streaming und Token-Zählung

In Situationen mit umfangreichen Datenmengen kann die Verarbeitungs- und Ausgabezeit der Ergebnisse erheblich sein. 
Um dies zu umgehen, ist es zweckmäßig, die bereits verarbeiteten Daten kontinuierlich auszugeben. 
Darüber hinaus beinhaltet dieses Experiment eine Funktion, die die verarbeiteten Tokens zählt und diese Information ebenfalls ausgibt.
Dies wird durch die Vererbung der BaseCallbackHandler-Klasse ermöglicht.

In [ ]:
from langchain.callbacks import get_openai_callback
from langchain.callbacks.base import BaseCallbackManager, BaseCallbackHandler
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.outputs.llm_result import LLMResult

class MyCustomHandler(BaseCallbackHandler):
    tokenCount = 0  

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.tokenCount += 1
        print(token, end="")

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:        
        print(f"\nAnzahl token: {self.tokenCount}")

model = AzureChatOpenAI(
    azure_deployment = "gpt-4",
    streaming=True,
    callback_manager=BaseCallbackManager([
        MyCustomHandler()]),
)

with get_openai_callback() as cb:
    message = HumanMessage(
        content="Verfasse eine 50 Wörter umfassende Vorlesegeschichte für Kinder, in der es um eine kleine Kartoffel namens Pellemann geht, die einen Hügel hinunterrollt, um ihre Freunde zu suchen."
    )
    model.invoke([message])    

Einmal war eine kleine Kartoffel, namens Pellemann, die allein auf einem Hügel lebte. Eines Tages beschloss er, seine Freunde zu suchen. Also kullerte er tapfer den Hügel hinunter. Er lachte, hüpfte und rollte, bis er schließlich seine Freunde fand. Sie spielten glücklich den ganzen Tag.
Anzahl token: 90
